In [27]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import os
from haversine import haversine, Unit

## Data reading

In [28]:
# Get a list of files to process
source_folder = Path("./data_JC")

source_files = os.listdir(source_folder)

source_files

['JC-202201-citibike-tripdata.csv',
 'JC-202202-citibike-tripdata.csv',
 'JC-202203-citibike-tripdata.csv',
 'JC-202204-citibike-tripdata.csv',
 'JC-202205-citibike-tripdata.csv',
 'JC-202206-citibike-tripdata.csv',
 'JC-202207-citbike-tripdata.csv',
 'JC-202208-citibike-tripdata.csv',
 'JC-202209-citibike-tripdata.csv',
 'JC-202210-citibike-tripdata.csv',
 'JC-202211-citibike-tripdata.csv',
 'JC-202212-citibike-tripdata.csv',
 'JC-202301-citibike-tripdata.csv',
 'JC-202302-citibike-tripdata.csv',
 'JC-202303-citibike-tripdata.csv',
 'JC-202304-citibike-tripdata.csv',
 'JC-202305-citibike-tripdata.csv',
 'JC-202306-citibike-tripdata.csv',
 'JC-202307-citibike-tripdata.csv',
 'JC-202308-citibike-tripdata.csv',
 'JC-202309-citibike-tripdata.csv',
 'JC-202310-citibike-tripdata.csv',
 'JC-202311-citibike-tripdata.csv',
 'JC-202312-citibike-tripdata.csv',
 'JC-202401-citibike-tripdata.csv',
 'JC-202402-citibike-tripdata.csv',
 'JC-202403-citibike-tripdata.csv',
 'JC-202404-citibike-tripdata

In [29]:
# Read data from all CSV files and combine them into a single DataFrame

df = pd.DataFrame({})

for file in source_files:
    file_path = Path(f"./data_JC/{file}")

    df_file = pd.read_csv(file_path)

    df = pd.concat([df_file, df], ignore_index=True)

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CD89CCE98ED4A830,classic_bike,2024-05-27 13:48:06,2024-05-27 13:50:25,Madison St & 10 St,HB503,7 St & Monroe St,HB304,40.749943,-74.035865,40.746413,-74.037977,member
1,D6AE40DD78B49217,classic_bike,2024-05-22 20:10:50,2024-05-22 20:22:13,Madison St & 10 St,HB503,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.749943,-74.035865,40.736982,-74.027781,member
2,39FDE65AAE7A4A22,electric_bike,2024-05-22 16:53:00,2024-05-22 17:12:58,Clinton St & 7 St,HB303,Columbus Drive,JC014,40.745420,-74.033411,40.718355,-74.038914,member
3,B39B432803AA828B,electric_bike,2024-05-19 09:22:01,2024-05-19 09:30:02,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725227,-74.045349,40.718355,-74.038914,member
4,0B7EE5E35F5BC47D,electric_bike,2024-05-10 19:59:35,2024-05-10 20:10:13,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725218,-74.045337,40.718355,-74.038914,member


## Data cleansing

### Calculate duration, distance and average speed of all trips

In [30]:
# Calculate trip duration in seconds
df["Duration"] = (pd.to_datetime(df["ended_at"]) - pd.to_datetime(df["started_at"])).dt.seconds

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration
0,CD89CCE98ED4A830,classic_bike,2024-05-27 13:48:06,2024-05-27 13:50:25,Madison St & 10 St,HB503,7 St & Monroe St,HB304,40.749943,-74.035865,40.746413,-74.037977,member,139
1,D6AE40DD78B49217,classic_bike,2024-05-22 20:10:50,2024-05-22 20:22:13,Madison St & 10 St,HB503,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.749943,-74.035865,40.736982,-74.027781,member,683
2,39FDE65AAE7A4A22,electric_bike,2024-05-22 16:53:00,2024-05-22 17:12:58,Clinton St & 7 St,HB303,Columbus Drive,JC014,40.745420,-74.033411,40.718355,-74.038914,member,1198
3,B39B432803AA828B,electric_bike,2024-05-19 09:22:01,2024-05-19 09:30:02,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725227,-74.045349,40.718355,-74.038914,member,481
4,0B7EE5E35F5BC47D,electric_bike,2024-05-10 19:59:35,2024-05-10 20:10:13,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725218,-74.045337,40.718355,-74.038914,member,638


In [31]:
# Calculate trip distance (in miles)
def calc_distance(row):
    return haversine((row["start_lat"], row["start_lng"]), (row["end_lat"], row["end_lng"]), unit=Unit.MILES)

df["Distance"] = df.apply(lambda row: calc_distance(row), axis=1)

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,Distance
0,CD89CCE98ED4A830,classic_bike,2024-05-27 13:48:06,2024-05-27 13:50:25,Madison St & 10 St,HB503,7 St & Monroe St,HB304,40.749943,-74.035865,40.746413,-74.037977,member,139,0.267828
1,D6AE40DD78B49217,classic_bike,2024-05-22 20:10:50,2024-05-22 20:22:13,Madison St & 10 St,HB503,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.749943,-74.035865,40.736982,-74.027781,member,683,0.990480
2,39FDE65AAE7A4A22,electric_bike,2024-05-22 16:53:00,2024-05-22 17:12:58,Clinton St & 7 St,HB303,Columbus Drive,JC014,40.745420,-74.033411,40.718355,-74.038914,member,1198,1.892034
3,B39B432803AA828B,electric_bike,2024-05-19 09:22:01,2024-05-19 09:30:02,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725227,-74.045349,40.718355,-74.038914,member,481,0.582247
4,0B7EE5E35F5BC47D,electric_bike,2024-05-10 19:59:35,2024-05-10 20:10:13,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725218,-74.045337,40.718355,-74.038914,member,638,0.581303


In [32]:
# Calculate average speed
df["Average Speed"] = df["Distance"] / (df["Duration"] / 3600)

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,Distance,Average Speed
0,CD89CCE98ED4A830,classic_bike,2024-05-27 13:48:06,2024-05-27 13:50:25,Madison St & 10 St,HB503,7 St & Monroe St,HB304,40.749943,-74.035865,40.746413,-74.037977,member,139,0.267828,6.936551
1,D6AE40DD78B49217,classic_bike,2024-05-22 20:10:50,2024-05-22 20:22:13,Madison St & 10 St,HB503,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.749943,-74.035865,40.736982,-74.027781,member,683,0.990480,5.220683
2,39FDE65AAE7A4A22,electric_bike,2024-05-22 16:53:00,2024-05-22 17:12:58,Clinton St & 7 St,HB303,Columbus Drive,JC014,40.745420,-74.033411,40.718355,-74.038914,member,1198,1.892034,5.685578
3,B39B432803AA828B,electric_bike,2024-05-19 09:22:01,2024-05-19 09:30:02,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725227,-74.045349,40.718355,-74.038914,member,481,0.582247,4.357775
4,0B7EE5E35F5BC47D,electric_bike,2024-05-10 19:59:35,2024-05-10 20:10:13,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725218,-74.045337,40.718355,-74.038914,member,638,0.581303,3.280080


### Clean-up incorrect records

In [33]:
# Remove any trips that are below 60 seconds in length (potentially false starts or users trying to re-dock a bike to ensure it's secure)
# Keep all trips that above 60 in length. Also check that start date/time of the ride is always earlier than end date/time
df = df[(df["Duration"] > 60) & (df["started_at"] < df["ended_at"])]

df.sort_values("Duration", ascending=True).head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,Distance,Average Speed
379327,8DFCA041D72D839F,classic_bike,2023-12-05 17:58:18,2023-12-05 17:59:19,River St & 1 St,HB609,Hoboken Terminal - River St & Hudson Pl,HB102,40.737215,-74.028865,40.736068,-74.029127,member,61,0.080473,4.749212
1542154,42E2DDEC60C9AC46,classic_bike,2022-10-15 12:31:29,2022-10-15 12:32:30,Liberty Light Rail,JC052,Liberty Light Rail,JC052,40.711242,-74.055701,40.711242,-74.055701,member,61,0.000000,0.000000
972713,0E5870680BF6CE86,classic_bike,2023-06-18 14:21:08,2023-06-18 14:22:09,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hoboken Terminal - River St & Hudson Pl,HB102,40.735938,-74.030305,40.736068,-74.029127,member,61,0.062298,3.676585
1137647,4F4D02970F5F735C,classic_bike,2023-04-10 14:33:07,2023-04-10 14:34:08,City Hall,JC003,Grove St PATH,JC115,40.717732,-74.043845,40.719410,-74.043090,member,61,0.122462,7.227279
504101,1DFFD5C38C5C0C9F,classic_bike,2023-10-30 13:10:21,2023-10-30 13:11:22,City Hall,JC003,Grove St PATH,JC115,40.717732,-74.043845,40.719410,-74.043090,member,61,0.122462,7.227279


In [34]:
# Keep all trips that have Average Speed >= 0 (0 means start and end stations are exactly the same) and less than 50 mph (reasonable speed of bike in the city)

df = df[(df["Average Speed"] >= 0) & (df["Average Speed"] <= 50)]

df.sort_values("Average Speed", ascending=False).head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Duration,Distance,Average Speed
582741,B788FB34CEC57929,classic_bike,2023-09-14 21:22:08,2023-09-14 21:24:21,Heights Elevator,JC059,Riverview Park,JC057,40.717636,-74.043918,40.744319,-74.043991,member,133,1.843596,49.901847
759908,30E5AA15A0B6D876,classic_bike,2023-08-20 18:55:38,2023-08-20 18:58:27,Willow Ave & 12 St,HB505,12 St & Sinatra Dr N,HB201,40.733741,-74.062613,40.750604,-74.024020,member,169,2.332214,49.680289
581350,B4F37B583CF88F29,classic_bike,2023-09-11 02:14:45,2023-09-11 02:16:14,Paulus Hook,JC002,Exchange Pl,JC116,40.731850,-74.045728,40.716366,-74.034344,member,89,1.224721,49.539258
48058,4D026E9DED8E39CE,electric_bike,2024-05-01 16:42:27,2024-05-01 16:44:32,Clinton St & Newark St,HB409,Church Sq Park - 5 St & Park Ave,HB601,40.718212,-74.038418,40.742659,-74.032233,member,125,1.719885,49.532700
1012685,2B774D1A51008E37,classic_bike,2023-05-23 19:16:09,2023-05-23 19:21:33,Bergen St & Flatbush Ave,4281.08,Pacific St & Smith St,4446.03,40.735354,-74.046973,40.687881,-73.989393,member,324,4.455663,49.507366


## Data saving

In [35]:
# Drop unnecessary columns
df.drop(columns=["Duration", "Distance", "Average Speed"], inplace=True)

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CD89CCE98ED4A830,classic_bike,2024-05-27 13:48:06,2024-05-27 13:50:25,Madison St & 10 St,HB503,7 St & Monroe St,HB304,40.749943,-74.035865,40.746413,-74.037977,member
1,D6AE40DD78B49217,classic_bike,2024-05-22 20:10:50,2024-05-22 20:22:13,Madison St & 10 St,HB503,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.749943,-74.035865,40.736982,-74.027781,member
2,39FDE65AAE7A4A22,electric_bike,2024-05-22 16:53:00,2024-05-22 17:12:58,Clinton St & 7 St,HB303,Columbus Drive,JC014,40.745420,-74.033411,40.718355,-74.038914,member
3,B39B432803AA828B,electric_bike,2024-05-19 09:22:01,2024-05-19 09:30:02,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725227,-74.045349,40.718355,-74.038914,member
4,0B7EE5E35F5BC47D,electric_bike,2024-05-10 19:59:35,2024-05-10 20:10:13,Jersey & 6th St,JC027,Columbus Drive,JC014,40.725218,-74.045337,40.718355,-74.038914,member


In [36]:
df.to_csv("JC-citibike-tripdata.csv", index=False, header=True)